In [1]:
import cv2                                         # cv2 is a module to import the image manipulation functions

import numpy as np              # Numerical Python is a Python library used to perform  mathematical operations on ndarrays

def canny(img):
    if img is None:
        cap.release()
        cv2.destroyAllWindows()
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # cv2.cvtColor() method is used to convert an image from one color space to another.
    blur = cv2.GaussianBlur(gray,(5, 5),0)
    canny = cv2.Canny(blur, 50, 150)
    #Canny() method is used to detect the edges in an image.
    return canny

def region_of_interest(canny):
    height = canny.shape[0]
    width = canny.shape[1]
    mask = np.zeros_like(canny)                          # Return an array of zeros with the same shape as a given array
    triangle = np.array([[
    (200, height),
    (800, 350),
    (1200, height),]], np.int32)
    cv2.fillPoly(mask, triangle, 255)                    # masking all the area which does not comes under triangle
    masked_image = cv2.bitwise_and(canny, mask)          # merging both images to get a cropped canny image
    return masked_image

def houghLines(cropped_canny):                                    # returns extreme points
    return cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, 
        np.array([]), minLineLength=40, maxLineGap=5)

def addWeighted(frame, line_image):                               # increse the thickness of detected lanes
    return cv2.addWeighted(frame, 0.8, line_image, 1, 1)
 
def display_lines(img,lines):                                     # displays lines on frames
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),10)
    return line_image
 
def make_points(image, line):
# this function defines the slope of the line based on the image sizes and assumes the size of the line
# and finds the slope of the image with the size given.
    print(line)
    slope, intercept = line
    y1 = int(image.shape[0])
    y2 = int(y1*3.5/5)      
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]
 
def average_slope_intercept(image, lines):
    left_fit    = []
    right_fit   = []
    if lines is None:
        return None
    for line in lines:
        for x1, y1, x2, y2 in line:
            fit = np.polyfit((x1,x2), (y1,y2), 1)            # it is used to minimize the squared error
            slope = fit[0]
            intercept = fit[1]
            if slope < 0: 
                left_fit.append((slope, intercept))
            else:
                right_fit.append((slope, intercept))
    left_fit_average  = np.average(left_fit, axis=0)   #averaging is performed on all of the axes specified in the tuple of ints
    right_fit_average = np.average(right_fit, axis=0)
    left_line  = make_points(image, left_fit_average)
    right_line = make_points(image, right_fit_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines

cap = cv2.VideoCapture("test1.mp4")             # method to capture the video
while(cap.isOpened()):
    #It returns True or False based on whether or not our cap object has started capturing the frames
    _, frame = cap.read()                                     #cap.read() function read the frames from our video.
    canny_image = canny(frame)                                #passing frames to canny function
    cropped_canny = region_of_interest(canny_image)           #passing canny_images to region of interest function
    cv2.imshow("Cropped_canny",cropped_canny)                 #cv2.imshow() method is used to display an image in a window.
    lines = houghLines(cropped_canny)                         #passing cropped_canny frames to houghLines function
    averaged_lines = average_slope_intercept(frame, lines)    #passing frames and extreme points
    line_image = display_lines(frame, averaged_lines)         #passing frames and modified points to display_lines function
    combo_image = addWeighted(frame, line_image)              #passing frames and detected lane image
    cv2.imshow("Result", combo_image)                         #displaying the image in a window
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
    #cv2.waitkey(1) will show a frame for at least 1 ms only and switches to show other frames, thus forming a video
        break

cap.release()
# Once our work with the video is done, it is required that we release the resources that we have initialized for our code

cv2.destroyAllWindows()
# destroyAllWindows() function allows users to destroy or close all windows

[-7.39092047e-01  9.56558669e+02]
[   0.96912693 -326.11361865]
[-7.20197523e-01  9.44087235e+02]
[   0.88360338 -260.09780224]
[-7.41571342e-01  9.58592524e+02]
[   1.01900096 -363.80835906]
[-7.54956368e-01  9.68189588e+02]
[   0.95561167 -312.06729569]
[-7.52921512e-01  9.65621890e+02]
[   0.94618066 -306.51209976]
[-7.68417636e-01  9.76459726e+02]
[   0.98814069 -331.91615622]
[-7.43738019e-01  9.60387606e+02]
[   0.86394774 -242.94906994]
[-7.34002696e-01  9.53182082e+02]
[   0.92070415 -290.96544176]
[-7.63711478e-01  9.74111078e+02]
[   0.93559709 -299.54296054]
[-7.60359550e-01  9.67791686e+02]
[   0.92911904 -287.36297961]
[-7.55045032e-01  9.59343744e+02]
[   0.89706681 -264.78307692]
[-7.90758043e-01  9.87729614e+02]
[   0.93994181 -291.27495212]
[-7.56724696e-01  9.62179113e+02]
[   0.96898012 -308.9795075 ]
[-7.92265832e-01  9.85952040e+02]
[   0.96543192 -305.97311871]
[-7.95097271e-01  9.85827654e+02]
[   0.9766331  -301.99059415]
[-8.11869734e-01  9.93021075e+02]
[   0.